In [10]:
import os
print(os.listdir())
from google.colab import files
uploaded = files.upload()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from xgboost import XGBRegressor
df = pd.read_csv("timing_performance_dataset.csv")

# Load dataset
df = pd.read_csv("timing_performance_dataset.csv")

# Identify the correct target column
print("Dataset Columns:", df.columns)  # Print column names to identify target
target_column = "Timing_Violation_Probability"  # Replace with the correct target column

if target_column not in df.columns:
    raise KeyError(f"Column '{target_column}' not found in dataset. Available columns: {df.columns}")

# Handle missing values
df.fillna(df.mean(), inplace=True)

X = df.drop(columns=[target_column])
y = df[target_column]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the dataset
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# ✅ Define the Neural Network Model
model = Sequential([
    Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.2),
    Dense(32, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.2),
    Dense(1)
])

# Compile the model
optimizer = Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# ✅ Train the model and store training history
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=50,
                    batch_size=32,
                    callbacks=[early_stopping, reduce_lr])

# Evaluate the model
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {test_mae}")

# ✅ Plot Training vs. Validation Loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training vs. Validation Loss')
plt.show()

# ✅ Alternative Model: XGBoost
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.05, random_state=42)
xgb_model.fit(X_train, y_train)

# Evaluate XGBoost model
xgb_predictions = xgb_model.predict(X_test)
xgb_mae = np.mean(np.abs(y_test - xgb_predictions))
print(f"XGBoost MAE: {xgb_mae}")
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Use correct variable names
y_true = y_test  # Actual labels from test data
y_pred = model.predict(X_test)  # Model predictions

# Calculate performance metrics
mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
r2 = r2_score(y_true, y_pred)

# Print results
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R² Score: {r2:.4f}")
y_pred = y_pred.flatten()  # Ensure correct shape


['.config', 'timing_performance_dataset (2).csv', 'timing_performance_dataset.csv', 'timing_performance_dataset (1).csv', 'sample_data']


KeyboardInterrupt: 